In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import losses
from tensorflow.keras import metrics

import matplotlib.pyplot as plt
import numpy

2023-05-29 10:06:53.901335: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-29 10:06:54.379657: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-29 10:06:54.381461: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-29 10:06:56.910891: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
data_generator = ImageDataGenerator(rescale=1.0/255., zoom_range=0.2, rotation_range=15,
                                             width_shift_range=0.05, height_shift_range=0.05)

In [3]:
DIRECTORY_TRAIN = "./Covid19-dataset/train"
DIRECTORY_TEST = "./Covid19-dataset/test"
CLASS_MODE = "categorical"
COLOR_MODE = "grayscale"
TARGET_SIZE = (256, 256)
BATCH_SIZE = 32

In [4]:
training_iterator = data_generator.flow_from_directory(DIRECTORY_TRAIN, class_mode=CLASS_MODE, color_mode=COLOR_MODE, 
                                                               target_size=TARGET_SIZE, batch_size=BATCH_SIZE)

Found 251 images belonging to 3 classes.


In [5]:
sample_batch_input, sample_batch_labels = training_iterator.next()

In [6]:
print(sample_batch_input.shape, sample_batch_labels.shape)

(32, 256, 256, 1) (32, 3)


In [7]:
validation_iterator = data_generator.flow_from_directory(DIRECTORY_TEST, class_mode=CLASS_MODE, color_mode=COLOR_MODE, 
                                                               target_size=TARGET_SIZE, batch_size=BATCH_SIZE)

Found 66 images belonging to 3 classes.


In [8]:
sample_batch_input, sample_batch_labels = validation_iterator.next()

In [9]:
print(sample_batch_input.shape, sample_batch_labels.shape)

(32, 256, 256, 1) (32, 3)


In [10]:
print(type(sample_batch_input), type(sample_batch_labels))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [11]:
def model_design():
    model = Sequential(name="classification_model")
    model.add(Input(shape=(256, 256, 1)))
    model.add(layers.Conv2D(2, 5, strides=3, padding="valid", activation="relu"))
    model.add(layers.MaxPooling2D(pool_size=(5, 5), strides=(5, 5)))
    model.add(layers.Conv2D(4, 3, strides=1, padding="valid", activation="relu"))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))) 
    model.add(layers.Flatten())
    model.add(layers.Dense(3, activation="softmax"))
    opt = Adam(learning_rate=0.01)
    model.compile(optimizer=opt, loss=losses.CategoricalCrossentropy() , metrics=[metrics.CategoricalAccuracy(), metrics.AUC()])
    return model

In [12]:
model = model_design()
print(model.summary())

Model: "classification_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 84, 84, 2)         52        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 2)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 4)         76        
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 4)          0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 196)               0         
                                                                 
 dense (Dense)               (None, 3)        

In [14]:
model.fit(training_iterator, steps_per_epoch = training_iterator.samples/BATCH_SIZE, epochs=30,
          validation_data = validation_iterator, validation_steps = validation_iterator.samples/BATCH_SIZE)

Epoch 1/30


2023-05-29 10:08:16.719256: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


8/7 [==============================] - ETA: 0s - loss: 0.5058 - categorical_accuracy: 0.8008 - auc: 0.9325

2023-05-29 10:08:19.244244: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


7/7 [==============================] - 3s 381ms/step - loss: 0.5058 - categorical_accuracy: 0.8008 - auc: 0.9325 - val_loss: 0.5968 - val_categorical_accuracy: 0.8030 - val_auc: 0.9079
Epoch 2/30
7/7 [==============================] - 3s 347ms/step - loss: 0.4844 - categorical_accuracy: 0.8367 - auc: 0.9376 - val_loss: 0.4870 - val_categorical_accuracy: 0.8333 - val_auc: 0.9346
Epoch 3/30
7/7 [==============================] - 3s 333ms/step - loss: 0.5028 - categorical_accuracy: 0.8088 - auc: 0.9314 - val_loss: 0.4483 - val_categorical_accuracy: 0.8333 - val_auc: 0.9518
Epoch 4/30
7/7 [==============================] - 3s 329ms/step - loss: 0.4179 - categorical_accuracy: 0.8486 - auc: 0.9511 - val_loss: 0.6104 - val_categorical_accuracy: 0.6818 - val_auc: 0.8978
Epoch 5/30
7/7 [==============================] - 3s 332ms/step - loss: 0.4611 - categorical_accuracy: 0.8406 - auc: 0.9417 - val_loss: 0.5920 - val_categorical_accuracy: 0.7121 - val_auc: 0.9023
Epoch 6/30
7/7 [===============